<p align="center"><img alt="udeA logo" height="150px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true"></p><h1><font color='0B5345'> <center>

Probabilidad y estadística

 </center></font></h1>
<h2><font color='0B5345'> <center>
Tercer Proyecto: Aprendizaje Supervisado</center></font></h2>
<font  face="Courier New" size="3">

<p3><center><b><font color='0B5345' face="Lucida Calligraphy,Comic Sans MS,Lucida Console" size="5">Universidad de Antioquia - Bioingeniería</font></b> </center></p3>

### **MODELO DE CLASIFICACIÓN**

### **ANÁLISIS EXPLORATORIO DE DATOS**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df_cancer = pd.read_csv('data_cancer.csv', sep=";")
df_cancer

Intercambiamos;

Benigno (B) = 0

Maligno (M) = 1

In [ ]:
df_cancer.replace(regex=r'B', value=0, inplace=True)
df_cancer.replace(regex=r'M', value=1, inplace=True)

In [ ]:
df_cancer.info()

In [ ]:
df_cancer.describe().T

En una inspección rápida, notamos que existen datos con valores iguales a 0. Estos datos no pueden existir ya que expresan medidas de volumen.

In [ ]:
for column in df_cancer.columns:
    zero_values = (df_cancer[column] == 0).sum()
    if zero_values != 0:
        print(column, zero_values)

Viendo la poca cantidad de datos, decidimos quitarlos del dataset. Diagnosis no va a ser afectado.

In [ ]:
df_cancer = df_cancer[(df_cancer.drop('diagnosis', axis=1) != 0).all(axis=1)]
len(df_cancer)

In [ ]:
df_cancer.describe().T

Para facilitar la visualización de los datos aplicamos z-score a todas las columnas

In [ ]:
df_cancer_z = df_cancer.drop(['diagnosis'], axis=1)
df_cancer_z = (df_cancer_z - df_cancer_z.mean()) / df_cancer_z.std()

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(15, 45))

# Histogramas

def hist_cancer(columna, ax1, name):
    for a in columna:
        sns.histplot(data=df_cancer_z, x=a, bins=20, kde=True, ax=ax1, label=a)
    ax1.legend()
    ax1.set_title(name)
    ax1.set_xlabel('')
    ax1.set_ylabel('')

radio = ['radius_mean', 'radius_se', 'radius_worst']
textura = ['texture_mean', 'texture_se', 'texture_worst']
perimetro = ['perimeter_mean', 'perimeter_se', 'perimeter_worst']
area = ['area_mean', 'area_se', 'area_worst']
suavidad = ['smoothness_mean', 'smoothness_se', 'smoothness_worst']
compactacion = ['compactness_mean', 'compactness_se', 'compactness_worst']
concavidad = ['concavity_mean', 'concavity_se', 'concavity_worst']
pts_concavidad = ['concave points_mean', 'concave points_se', 'concave points_worst']
simetria = ['symmetry_mean', 'symmetry_se', 'symmetry_worst']
dim_frac = ['fractal_dimension_mean', 'fractal_dimension_se', 'fractal_dimension_worst']

hist_cancer(radio, axs[0, 0], 'Radio')
hist_cancer(textura, axs[1, 0], 'Textura')
hist_cancer(perimetro, axs[2, 0], 'Perimetro')
hist_cancer(area, axs[3, 0], 'Area')
hist_cancer(suavidad, axs[4, 0], 'Suavidad')
hist_cancer(compactacion, axs[5, 0], 'Compactación')
hist_cancer(concavidad, axs[6, 0], 'Concavidad')
hist_cancer(pts_concavidad, axs[7, 0], 'Puntos de concavidad')
hist_cancer(simetria, axs[8, 0], 'Simetria')
hist_cancer(dim_frac, axs[9, 0], 'Dim. Fractal')

sns.boxplot(data=df_cancer_z[radio], ax=axs[0, 1])
sns.boxplot(data=df_cancer_z[textura], ax=axs[1, 1])
sns.boxplot(data=df_cancer_z[perimetro], ax=axs[2, 1])
sns.boxplot(data=df_cancer_z[area], ax=axs[3, 1])
sns.boxplot(data=df_cancer_z[suavidad], ax=axs[4, 1])
sns.boxplot(data=df_cancer_z[compactacion], ax=axs[5, 1])
sns.boxplot(data=df_cancer_z[concavidad], ax=axs[6, 1])
sns.boxplot(data=df_cancer_z[pts_concavidad], ax=axs[7, 1])
sns.boxplot(data=df_cancer_z[simetria], ax=axs[8, 1])
sns.boxplot(data=df_cancer_z[dim_frac], ax=axs[9, 1])

Remoción de datos atípicos

In [ ]:
def remove_outliers_iqr(data):
    
    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
    
    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remove outliers
    data = np.where(data>upper_bound, upper_bound, np.where(data<lower_bound,lower_bound,data))
    return data[(data >= lower_bound) & (data <= upper_bound)]

In [ ]:
for i in df_cancer.columns:
    df_cancer[i] = remove_outliers_iqr(df_cancer[i])

len(df_cancer)

In [ ]:
df_cancer

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_cancer.corr(),fmt='.2g',annot=True)
plt.show()

> Una primera observación a destacar es la fuerte correlación del radio, perímetro y área de la imagen, debido a que el área es la que aporta más información, elegimos las columnas relacionadas para nuestro modelo.


> El segundo aspecto a destacar son las correlaciones entre las medias de todos los datos y su peor valor registrado, si bien no tan significatico como el punto anterior, sigue teniendo una influencia significativa con valores de hasta 0.8. Se decidió por la media por aportar más información.

Así despues de una purga de columnas obtenemos:

In [ ]:
df_cancer_test =  df_cancer[['diagnosis', 
                            'area_mean', 'area_se',
                            'smoothness_mean', 'smoothness_se',
                            'compactness_mean', 'compactness_se',
                            'concavity_mean', 'concavity_se',
                            'concave points_mean', 'concave points_se',
                            'symmetry_mean', 'symmetry_se',
                            'fractal_dimension_mean', 'fractal_dimension_se']]

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_cancer_test.corr(),fmt='.2g',annot=True)
plt.show()

En este segundo análisis de datos, notemos que la concavidad y los puntos de concavidad siguen fuertemente relacionados entre sí, elegimos los puntos de concavidad debido a que presenta una mayor correlación con el diagnóstico.

Lo mismo sucede con la compactación, por este motivo se elige solo los puntos de concavidad.

In [ ]:
df_cancer_test =  df_cancer[['diagnosis', 
                            'area_mean', 'area_se',
                            'smoothness_mean', 'smoothness_se',
                            'concave points_mean', 'concave points_se',
                            'symmetry_mean', 'symmetry_se',
                            'fractal_dimension_mean', 'fractal_dimension_se']]

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_cancer_test.corr(),fmt='.2g',annot=True)
plt.show()

In [ ]:
df_cancer_test

**Variables**:

**Explicada (Y)**: 
- Diagnostico(tumor benigno o no maligno)

**Explicatorias (X)**: 
- area_mean 
- area_se
- smoothness_mean 
- smoothness_se
- concave points_mean
- concave points_se
- symmetry_mean 
- symmetry_se
- fractal_dimension_mean
- fractal_dimension_se

Importamos las librerias

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
x = df_cancer_test.drop(columns=['diagnosis'])

y = df_cancer_test['diagnosis']

Datos de entrenamiento

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

Entrenamiento del modelo